In [1]:
import pandas as pd
import numpy as np

from os import listdir
from os.path import isfile, join

### Helper Functions

In [2]:
def all_files(path):
    return [join(path, f) for f in listdir(path) if isfile(join(path, f))]

### Get file paths

In [3]:
data_path = join('..', '..', '..', 'Data')

pi_path  = join(data_path, 'KGW', '01_PI_Data')
gt_path  = join(data_path, 'KGW', '02_GT_Data')
dcs_path = join(data_path, 'KGW', '03_DCS_Data')
db_path  = join(data_path, 'kgw_data.h5')

In [4]:
pi_files  = all_files(pi_path)
gt_files  = all_files(gt_path)
dcs_files = all_files(dcs_path)

## Importing CSV data

Please ensure that the following functions are run only once to avoid multiple entries for the same data in the DB.

### Import PI data CSVs

In [5]:
def group_by(f, xs):
    from collections import defaultdict
    groups = defaultdict(list)    
    for x in xs:
        groups[f(x)].append(x)
    return groups.values()


def merge_all(frames, on="TIMESTAMP", how='inner'):
    if len(frames) == 1:
        return frames[0]
    
    from functools import reduce
    def merge(a, b): 
        pd.merge(a, b, on=on, how=how)
    return reduce(merge, frames)
        

def import_data(files, datetime_format='%d.%m.%Y %H:%M'):
    date_format = lambda x: pd.datetime.strptime(x, datetime_format)
    
    def parse(p):
        return pd.read_csv(p, 
                           parse_dates=['TIMESTAMP'], 
                           infer_datetime_format=True, 
                           date_parser=date_format, 
                           sep=';', 
                           engine='c', 
                           low_memory=False, 
                           na_values=["#NV", '?0,0', '?-17,0'], 
                           decimal=',', 
                           thousands='.', 
                           memory_map=True)
    
    def read(files):
        return pd.concat([parse(f) for f in files])

    def read_basic(files):
        return read(files)
    
    def read_extra(files):
        import re

        def name_index(file): 
            return int(re.search(r'#(\d+)', file).group(1))

        groups = group_by(name_index, files)           
        frames = [read(fs) for fs in groups]
  
        return merge_all(frames)
    
    basic_files = [f for f in files if "#" not in f]
    extra_files = [f for f in files if "#" in f]

    if extra_files:
        basic_df = read_basic(basic_files)
        extra_df = read_extra(extra_files)
        final_df = pd.merge(basic_df, extra_df, on='TIMESTAMP') 
        print(f'merging with extra data\nnumber of rows\nbasic: {len(basic_df)}\nextra: {len(extra_df)}\nfinal: {len(final_df)}\n-------------\ndiff:  {len(basic_df) - len(final_df)}')
        return final_df
    else:
        return read_basic(basic_files)

In [6]:
%%time
pi_df = import_data(pi_files)
pi_df = pi_df.convert_objects(convert_numeric=True)

merging with extra data
number of rows
basic: 403140
extra: 403140
final: 403140
-------------
diff:  0


/home/ab/Develop/envs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


CPU times: user 6min 14s, sys: 29.7 s, total: 6min 43s
Wall time: 52.7 s


### Import GT data CSVs

In [7]:
%%time
gt_df = import_data(gt_files)
gt_df = gt_df.convert_objects(convert_numeric=True)

/home/ab/Develop/envs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


CPU times: user 7.15 s, sys: 1.01 s, total: 8.16 s
Wall time: 4.28 s


### Import DCS data CSVs

In [8]:
%%time
dcs_df = import_data(dcs_files, datetime_format='%d.%m.%Y %H:%M:%S')
dcs_df = dcs_df.convert_objects(convert_numeric=True)

/home/ab/Develop/envs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


CPU times: user 14.6 s, sys: 717 ms, total: 15.3 s
Wall time: 5.09 s


## Impute missing time data

In [9]:
gt_dcs    = pd.merge(gt_df,  dcs_df, on='TIMESTAMP', how='outer')
gt_dcs_pi = pd.merge(gt_dcs, pi_df,  on='TIMESTAMP', how='outer')
final_df = gt_dcs_pi.sort_values('TIMESTAMP')

In [10]:
# cast 'on / of' columns to 1 = on and 0 = of
def get(s):
    value_dict = { 'aus': 0, 'ein': 1, 'nan': np.nan, 'bad input': np.nan }   
    k = str(s).strip().lower()
    if k in value_dict:
        return value_dict[k]
    else:
        print(f'unkown value: --{s}--')
        return np.nan

for c in ['120 A0 LAC60 AP001XB01', '120 A0 LAC65 AP001XB01', '120 A0 LAC70 AP001XB01', '120 A0 LAC75 AP001XB01', '120 A0 LAC80 AP001XB01']:
    final_df[c] = final_df[c].apply(get)
    
    
# ensure all data columns are 32bit floats instead of 64bit
for c in final_df.columns[1:]:
    final_df[c] = final_df[c].astype('float32')

In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 416315 entries, 18063 to 40252
Columns: 746 entries, TIMESTAMP to 053 A6 MAY50 GH010ZXQ51
dtypes: datetime64[ns](1), float32(745)
memory usage: 1.2 GB


## Store Data Frames in HDF5 DB

In [12]:
db_path = join('..', '..', '..', 'Data', 'kgw_data.h5')

with pd.HDFStore(db_path) as DB:
    DB.put(key='pi_data',  value=pi_df)
    DB.put(key='gt_data',  value=gt_df)
    DB.put(key='dcs_data', value=dcs_df)
    DB.put(key='all_data', value=final_df)

## Read Data Frames from HDF5 DB

The following snippet should used in the notebook used for analysing the data.

In [13]:
%%time
from os.path import join
import pandas as pd

db_path = join('..', '..', '..', 'Data', 'kgw_data.h5')

pi_df  = pd.read_hdf(db_path, key='pi_data',  format='t')
gt_df  = pd.read_hdf(db_path, key='gt_data',  format='t')
dcs_df = pd.read_hdf(db_path, key='dcs_data', format='t')
all_df = pd.read_hdf(db_path, key='all_data', format='t')

CPU times: user 87 ms, sys: 1.11 s, total: 1.19 s
Wall time: 1.19 s


In [14]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 416315 entries, 18063 to 40252
Columns: 746 entries, TIMESTAMP to 053 A6 MAY50 GH010ZXQ51
dtypes: datetime64[ns](1), float32(745)
memory usage: 1.2 GB


In [15]:
all_df.dtypes

TIMESTAMP                       datetime64[ns]
AN_AGb_Acceleration                    float32
AN_AGB_Accel_Band0                     float32
AN_AGB_Accel_Band1                     float32
AN_AGB_Accel_Band2                     float32
AN_AGB_Accel_Band3                     float32
AN_AGB_Hours                           float32
AN_AGB_Start_Count                     float32
AN_Air_Supply_Pressure                 float32
AN_Ambient_Air_Humidity                float32
AN_Ambient_Air_Temperature             float32
AN_BDEW_Bus_Volt                       float32
AN_BDEW_Bus_Volt_Pcnt                  float32
AN_BDEW_Delta_P                        float32
AN_BDEW_High_Freq_P_Limit              float32
AN_BDEW_Nom_Util_Volt                  float32
AN_BDEW_Pwr_Lim                        float32
AN_BDEW_P_Ist                          float32
AN_BDEW_Rated_Power                    float32
AN_BDEW_Signals_P_Limit                float32
AN_BDEW_Signals_P_Limit_Pcnt           float32
AN_BDEW_Sync_